### Funcionou (5% da anterior, bonificações sobre movimentos corretos e modo roleta)

In [21]:
import random
import numpy as np
import pandas as pd

# Constantes de limites (estas precisam ser definidas em seu código)
KM = 10  # Exemplo de valor para KM, defina conforme necessário
MIN_GM = 1  # Exemplo de valor para MIN_GM, defina conforme necessário
MAX_HEEL = 5  # Exemplo de valor para MAX_HEEL, defina conforme necessário
MAX_TRIM = 5  # Exemplo de valor para MAX_TRIM, defina conforme necessário

def gerar_cromossomo():
    cromossomo = []
    
    # Gera posições iniciais (0-29)
    posicoes_iniciais = list(range(30))
    random.shuffle(posicoes_iniciais)
    cromossomo.extend(posicoes_iniciais)
    
    # Gera posições finais (0-29)
    posicoes_finais = list(range(30))
    random.shuffle(posicoes_finais)
    cromossomo.extend(posicoes_finais)
    
    # Gera ordem de movimentação (0-29)
    ordem_movimentacao = list(range(30))
    random.shuffle(ordem_movimentacao)
    cromossomo.extend(ordem_movimentacao)
    
    return cromossomo

def cromossomo_para_dataframe(cromossomo):
    container = list(range(30))
    posicao_inicial = cromossomo[:30]
    posicao_final = cromossomo[30:60]
    ordem_movimentacao = cromossomo[60:90]
    
    df = pd.DataFrame({
        'Container': container,
        'Posição Inicial': posicao_inicial,
        'Posição Final': posicao_final,
        'Ordem de Movimentação': ordem_movimentacao
    })
        
    df = df.sort_values(by='Ordem de Movimentação')
    
    return df

def inicializar_populacao(tamanho_populacao):
    populacao = []
    for _ in range(tamanho_populacao):
        cromossomo = gerar_cromossomo()
        populacao.append(cromossomo)
    return populacao

def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

def calcular_estabilidade(containers):
    if len(containers) < 2:
        return True, 0, 0, 0, 0  # Retorna valores padrão para listas vazias ou com apenas um contêiner
    
    # Calcula o peso total dos contêineres até o penúltimo
    peso_total = sum(peso for peso, _, _, _ in containers[:-1])
    if peso_total == 0:
        return True, 0, 0, 0, 0  # Retorna valores padrão se peso_total for zero
    
    # Calcula o centro de gravidade no eixo Z (vertical) até o penúltimo
    centro_gravidade_z = sum(peso * z for peso, _, _, z in containers[:-1]) / peso_total
    # Calcula o centro de gravidade no eixo X (horizontal) até o penúltimo
    centro_gravidade_x = sum(peso * x for peso, x, _, _ in containers[:-1]) / peso_total
    # Calcula o centro de gravidade no eixo Y (longitudinal) até o penúltimo
    centro_gravidade_y = sum(peso * y for peso, _, y, _ in containers[:-1]) / peso_total
    # Pega os valores do último contêiner
    peso_ultimo, x_ultimo, y_ultimo, z_ultimo = containers[-1]
    
    # Calcula as diferenças entre os pontos geométricos e os eixos do último contêiner usando valores absolutos
    diff_x = abs(centro_gravidade_x - x_ultimo)
    diff_y = abs(centro_gravidade_y - y_ultimo)
    diff_z = abs(centro_gravidade_z - z_ultimo)
    
    # Soma as diferenças
    soma_diferencas = diff_x + diff_y + diff_z
    
    # Calcula a altura metacêntrica (GM)
    GM = KM - centro_gravidade_z
    # Calcula a inclinação lateral (heel)
    heel = sum(peso * x for peso, x, _, _ in containers) / (peso_total + peso_ultimo) / 10
    # Calcula a inclinação longitudinal (trim)
    trim = sum(peso * y for peso, _, y, _ in containers) / (peso_total + peso_ultimo) / 10
    # Verifica se os parâmetros de estabilidade estão dentro dos limites aceitáveis
    if GM < MIN_GM or abs(heel) > MAX_HEEL or abs(trim) > MAX_TRIM:
        return False, GM, heel, trim, soma_diferencas
    
    # Retorna a estabilidade, os parâmetros calculados, o ponto geométrico e a soma das diferenças
    return True, GM, heel, trim, soma_diferencas

# Função modificada para incluir cálculo de estabilidade
def preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    cal_movimentos = 0
    cal_estabilidade = 0
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    containers = []
    
    errou = 0
    pode_mover = True
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        pos_inicial = linha['Posição Inicial'] 
        pos_final = linha['Posição Final']         
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        if (errou == 0):            
            # Verificar se podemos remover do porto preenchido
            if andar_ini < 1:
                if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                    errou = 1
                    pode_mover = False
            
            # Verificar se podemos adicionar à balsa
            if andar_fin > 0:
                if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                    errou = 1
                    pode_mover = False

            # Verifica se o lugar está ocupado
            if balsa[andar_fin, x_fin, y_fin] == 1:
                errou = 1
                pode_mover = False
                    
            if pode_mover:
                # Atualizar porto_preenchido e balsa
                porto_preenchido[andar_ini, x_ini, y_ini] = 0
                balsa[andar_fin, x_fin, y_fin] = 1
                
                cal_movimentos += 2  # alturas subir e descer
                cal_movimentos += 0 if x_ini == x_fin else 1  # verificação x
                cal_movimentos += 0 if y_ini == y_fin else 1  # verificação y
                
                # Adicionar contêiner à lista
                peso_container = 10  # Supondo peso fixo de 10 toneladas por contêiner
                containers.append((peso_container, x_fin, y_fin, 1))  # Altura simplificada

                estavel, GM, heel, trim, valor_estabilidade = calcular_estabilidade(containers)

                # Penalização para containers em andar superior sem preenchimento inferior completo
                if andar_fin > 0 and np.any(porto_preenchido[andar_fin - 1, :, :] == 1):
                    cal_estabilidade += 1000
                    
                if not estavel:
                    print(f"Inestável após mover contêiner {container}: GM={GM}, Heel={heel}, Trim={trim}")
                    errou = 1
                    pode_mover = False
                    cal_estabilidade += 100
                else:
                    cal_estabilidade += valor_estabilidade
            else:
                cal_movimentos += 1000
                cal_estabilidade += 100
        else:
            cal_movimentos += 1000
            cal_estabilidade += 100

    beta = 3
    fit = cal_movimentos + (beta * cal_estabilidade)
    return fit

def selecao_roleta(populacao, fitness):
    soma_fitness = sum(fitness)
    pick = random.uniform(0, soma_fitness)
    atual = 0
    for cromossomo, fit in zip(populacao, fitness):
        atual += fit
        if atual >= pick:
            return cromossomo

def cruzamento_pmx(pai1, pai2, inicio, fim):
    tamanho = fim - inicio + 1
    filho1 = pai1[:inicio] + [-1] * tamanho + pai1[fim+1:]
    filho2 = pai2[:inicio] + [-1] * tamanho + pai2[fim+1:]
    
    mapeamento1 = pai1[inicio:fim+1]
    mapeamento2 = pai2[inicio:fim+1]
    
    # Copiar o segmento
    filho1[inicio:fim+1] = mapeamento2
    filho2[inicio:fim+1] = mapeamento1
    
    # Resolver conflitos para filho1
    for i in range(inicio):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    for i in range(fim+1, len(pai1)):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    # Resolver conflitos para filho2
    for i in range(inicio):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    for i in range(fim+1, len(pai2)):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    return filho1, filho2

def cruzamento(pai, mae):
    filho1, filho2 = pai[:], mae[:]
    
    ponto_corte1 = random.randint(0, 29)
    ponto_corte2 = random.randint(0, 29)
    inicio, fim = min(ponto_corte1, ponto_corte2), max(ponto_corte1, ponto_corte2)
    filho1[:30], filho2[:30] = cruzamento_pmx(pai[:30], mae[:30], inicio, fim)
    
    ponto_corte1 = random.randint(30, 59)
    ponto_corte2 = random.randint(30, 59)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 30, max(ponto_corte1, ponto_corte2) - 30
    filho1[30:60], filho2[30:60] = cruzamento_pmx(pai[30:60], mae[30:60], inicio, fim)
    
    ponto_corte1 = random.randint(60, 89)
    ponto_corte2 = random.randint(60, 89)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 60, max(ponto_corte1, ponto_corte2) - 60
    filho1[60:90], filho2[60:90] = cruzamento_pmx(pai[60:90], mae[60:90], inicio, fim)
    
    return filho1, filho2

def mutacao(cromossomo, taxa_mutacao=0.01):
    if random.random() < taxa_mutacao:
        part = random.randint(0, 2)
        idx1, idx2 = random.sample(range(30), 2)
        if part == 0:
            cromossomo[idx1], cromossomo[idx2] = cromossomo[idx2], cromossomo[idx1]
        elif part == 1:
            cromossomo[30+idx1], cromossomo[30+idx2] = cromossomo[30+idx2], cromossomo[30+idx1]
        else:
            cromossomo[60+idx1], cromossomo[60+idx2] = cromossomo[60+idx2], cromossomo[60+idx1]
    return cromossomo

def algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao):
    populacao = inicializar_populacao(tamanho_populacao)
    fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
    
    melhor_fit = min(fitness)
    repeticoes = 0
    geracao = 0

    while repeticoes < 100:
        nova_populacao = []
        while len(nova_populacao) < tamanho_populacao:
            pai = selecao_roleta(populacao, fitness)
            mae = selecao_roleta(populacao, fitness)
            
            filho1, filho2 = cruzamento(pai, mae)
            filho1 = mutacao(filho1, taxa_mutacao)
            filho2 = mutacao(filho2, taxa_mutacao)
            
            nova_populacao.append(filho1)
            nova_populacao.append(filho2)
        
        # Combinar 5% melhor da geração antiga com a melhor nova geração
        populacao.extend(nova_populacao)
        fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
        
        # Ordenar pela fitness e selecionar os melhores
        populacao_ordenada = sorted(zip(populacao, fitness), key=lambda x: x[1])
        
        melhor_5_porcento = int(tamanho_populacao * 0.05)
        nova_metade = tamanho_populacao - melhor_5_porcento
        
        populacao = [cromossomo for cromossomo, _ in populacao_ordenada[:melhor_5_porcento]] + [cromossomo for cromossomo, _ in populacao_ordenada[melhor_5_porcento:melhor_5_porcento+nova_metade]]
        fitness = [fit for _, fit in populacao_ordenada[:melhor_5_porcento]] + [fit for _, fit in populacao_ordenada[melhor_5_porcento:melhor_5_porcento+nova_metade]]
        
        melhor_fit_atual = min(fitness)
        if melhor_fit_atual == melhor_fit:
            repeticoes += 1
        else:
            melhor_fit = melhor_fit_atual
            repeticoes = 0

        geracao += 1
        print(f"Geração {geracao}: Melhor fitness = {melhor_fit} (Repetiu {repeticoes} vezes)")
    
    melhor_cromossomo = populacao[fitness.index(melhor_fit)]
    return melhor_cromossomo, melhor_fit


# Configurações
tamanho_populacao = 100
geracoes = 200
taxa_mutacao = 0.1

melhor_cromossomo, melhor_fit = algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao)
print(f"Melhor cromossomo encontrado: {melhor_cromossomo}")
print(f"Melhor fitness: {melhor_fit}")


Geração 1: Melhor fitness = 35129.5 (Repetiu 1 vezes)
Geração 2: Melhor fitness = 35129.5 (Repetiu 2 vezes)
Geração 3: Melhor fitness = 35127.0 (Repetiu 0 vezes)
Geração 4: Melhor fitness = 35127.0 (Repetiu 1 vezes)
Geração 5: Melhor fitness = 35122.0 (Repetiu 0 vezes)
Geração 6: Melhor fitness = 35122.0 (Repetiu 1 vezes)
Geração 7: Melhor fitness = 35120.0 (Repetiu 0 vezes)
Geração 8: Melhor fitness = 35120.0 (Repetiu 1 vezes)
Geração 9: Melhor fitness = 35120.0 (Repetiu 2 vezes)
Geração 10: Melhor fitness = 34237.3 (Repetiu 0 vezes)
Geração 11: Melhor fitness = 34237.3 (Repetiu 1 vezes)
Geração 12: Melhor fitness = 34237.3 (Repetiu 2 vezes)
Geração 13: Melhor fitness = 34237.3 (Repetiu 3 vezes)
Geração 14: Melhor fitness = 33839.0 (Repetiu 0 vezes)
Geração 15: Melhor fitness = 33839.0 (Repetiu 1 vezes)
Geração 16: Melhor fitness = 33829.5 (Repetiu 0 vezes)
Geração 17: Melhor fitness = 32541.5 (Repetiu 0 vezes)
Geração 18: Melhor fitness = 32541.5 (Repetiu 1 vezes)
Geração 19: Melhor 

#### Verifica se ha numeros repetidos


In [22]:
def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

if verifica_repetidos(melhor_cromossomo):
    print("Valores repetidos encontrados!")
else:
    print("Não há valores repetidos.")

df_cromossomo = cromossomo_para_dataframe(melhor_cromossomo)
print("DataFrame do Cromossomo:")
print(df_cromossomo)


Não há valores repetidos.
DataFrame do Cromossomo:
    Container  Posição Inicial  Posição Final  Ordem de Movimentação
18         18               18              3                      0
11         11               20              5                      1
22         22               19              4                      2
4           4               22              1                      3
26         26               24             10                      4
6           6               21              6                      5
9           9                9              9                      6
5           5               26             11                      7
17         17               23              8                      8
0           0                7              7                      9
28         28               17              2                     10
27         27               15              0                     11
14         14                5             25       

### Simular Melhor cromomossomo

In [23]:
def Simular_preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []
    

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] 
        pos_final = linha['Posição Final'] 
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        # Verificar se podemos remover do porto preenchido
        pode_mover = True
        if andar_ini < 1:
            if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                pode_mover = False
        
        # Verificar se podemos adicionar à balsa
        if andar_fin > 0:
            if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                pode_mover = False
        
        if pode_mover:
            # Atualizar porto_preenchido e balsa
            porto_preenchido[andar_ini, x_ini, y_ini] = 0
            balsa[andar_fin, x_fin, y_fin] = 1
            
            print(f"Movendo container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")
            print("Estado atual do Porto Preenchido:")
            print(porto_preenchido)
            print("Estado atual da Balsa:")
            print(balsa)
            print("\n")
        else:
            print(f"não moveu container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")


Simular_preencher_porto(cromossomo_para_dataframe(melhor_cromossomo))

Movendo container 18 de (18)(18)(1, 1, 0) para (3)(3)(0, 1, 0)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [0 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 0 0]
  [1 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 11 de (20)(20)(1, 1, 2) para (5)(5)(0, 1, 2)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [0 1 0]
  [1 1 1]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 0 0]
  [1 0 1]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 22 de (19)(19)(1, 1, 1) para (4)(4)(0, 1, 1)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [0 0 0]
  [1 1 1]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 0 0]
  [1 1 1]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
